## HI!

This Jupyter notebook will be used to submit my IBM Data Science course capstone project. We are learning about location data and also utilising the knowledge that we have gathered over the previous modules. 

So lets get started!

In [3]:
#Firstly we need to import the libraries we are going to use
import pandas as pd
import numpy as np


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
